<a href="https://colab.research.google.com/github/JoelWekesa/Machine-and-Deep-Learning/blob/main/housingpricesprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, BaggingRegressor, VotingRegressor
from sklearn.svm import LinearSVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats.stats import pearsonr
from google.colab import drive

In [ ]:
pd.set_option("display.max_rows", 100)
sns.set_style("darkgrid")
sns.set(rc = {'figure.figsize': (12, 10)})

In [ ]:
drive.mount("/content/datasets")

Drive already mounted at /content/datasets; to attempt to forcibly remount, call drive.mount("/content/datasets", force_remount=True).


In [ ]:
path = "/content/datasets/MyDrive/datasets/house-prices-advanced-regression-techniques/train.csv"

In [ ]:
df = pd.read_csv(path)

In [ ]:
df = df.drop(columns = ["Id", "MiscFeature", "Fence", "PoolQC", "Alley"])

In [ ]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [ ]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
min_price, max_price = df["SalePrice"].quantile([0.001, 0.999])

In [ ]:
filt_min = (df["SalePrice"] < min_price)

In [ ]:
min_df = df.loc[filt_min]

In [ ]:
filt_max = (df["SalePrice"] > max_price)

In [ ]:
max_df = df.loc[filt_max]

In [ ]:
price_outliers = pd.concat([min_df, max_df])

In [ ]:
df = df.drop(price_outliers.index)

In [ ]:
encoder = LabelEncoder()
imputer = SimpleImputer()
hot = OneHotEncoder()
scaler = StandardScaler()

In [ ]:
for i in df.columns:
  if df[i].dtype != "int" and df[i].dtype != "float":
    df[i] = df[i].fillna(df[i].value_counts().idxmax())
    df[i] = encoder.fit_transform(df[i])

In [ ]:
X = df.drop(columns = ["SalePrice"])
X = imputer.fit_transform(X)

In [ ]:
y = df["SalePrice"]

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
train_X = scaler.fit_transform(train_X)

In [ ]:
test_X = scaler.transform(test_X)

In [ ]:
regressor = LinearRegression()

In [ ]:
regressor.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
predictions = regressor.predict(test_X)

In [ ]:
r2_score(test_y, predictions)

0.8647015537788565

In [ ]:
mean_squared_error(test_y, predictions)

968710363.4874146

In [ ]:
rfc = RandomForestRegressor()

In [ ]:
rfc.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
rfc_pred = rfc.predict(test_X)

In [ ]:
r2_score(test_y, rfc_pred)

0.8855504270140075

In [ ]:
ridge = RidgeCV()

In [ ]:
ridge.fit(train_X, train_y)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [ ]:
rid_pred = ridge.predict(test_X)

In [ ]:
r2_score(test_y, rid_pred)

0.8650200405393471

In [ ]:
lasso = LassoCV()

In [ ]:
lasso.fit(train_X, train_y)

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [ ]:
lasso_pred = lasso.predict(test_X)

In [ ]:
r2_score(test_y, lasso_pred)

0.8610023491718363

In [ ]:
boost = GradientBoostingRegressor()

In [ ]:
boost.fit(train_X, train_y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
boost_pred = boost.predict(test_X)

In [ ]:
r2_score(test_y, boost_pred) 

0.9059350148323178

In [ ]:
xboost = XGBRegressor()

In [ ]:
xboost.fit(train_X, train_y)

[09:52:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
xboost_pred = xboost.predict(test_X)

In [ ]:
r2_score(test_y, xboost_pred)

0.9012047474964119

In [ ]:
svr = LinearSVR()

In [ ]:
svr.fit(train_X, train_y)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [ ]:
svr_pred = svr.predict(test_X)

In [ ]:
r2_score(test_y, svr_pred)

-4.461201767790967

In [ ]:
bagging = BaggingRegressor()

In [ ]:
bagging.fit(train_X, train_y)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

In [ ]:
bagging_pred = bagging.predict(test_X)

In [ ]:
r2_score(test_y, bagging_pred)

0.8603520528455453

In [ ]:
estimators = [("xboost", xboost), ("boost", boost)]

In [ ]:
stacked = StackingRegressor(estimators=estimators, final_estimator=boost)
estims = [("stack", stacked), ("xboost", xboost), ("boost", boost)]
voted = VotingRegressor(estimators=estims)

In [ ]:
stacked.fit(train_X, train_y)
voted.fit(train_X, train_y)

[09:52:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:52:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

VotingRegressor(estimators=[('stack',
                             StackingRegressor(cv=None,
                                               estimators=[('xboost',
                                                            XGBRegressor(base_score=0.5,
                                                                         booster='gbtree',
                                                                         colsample_bylevel=1,
                                                                         colsample_bynode=1,
                                                                         colsample_bytree=1,
                                                                         gamma=0,
                                                                         importance_type='gain',
                                                                         learning_rate=0.1,
                                                                         max_delta_step=0,
              

In [ ]:
stacked_pred = stacked.predict(test_X)
voted_pred = voted.predict(test_X)

In [ ]:
r2_score(test_y, stacked_pred)

0.8917408965641279

In [ ]:
r2_score(test_y, voted_pred)

0.9051275287748691

In [ ]:
path2 = "/content/datasets/MyDrive/datasets/house-prices-advanced-regression-techniques/test.csv"

In [ ]:
tdf = pd.read_csv(path2)

In [ ]:
tedf = tdf.drop(columns = ["Id", "MiscFeature", "Fence", "PoolQC", "Alley"])

In [ ]:
for i in tedf.columns:
  if tedf[i].dtype != "int" and tedf[i].dtype != "float":
    tedf[i] = tedf[i].fillna(tedf[i].value_counts().idxmax())
    tedf[i] = encoder.fit_transform(tedf[i])

In [ ]:
tedf = imputer.fit_transform(tedf)

In [ ]:
tedf = scaler.transform(tedf)

In [ ]:
f_predictions = voted.predict(tedf)

In [ ]:
tdf = tdf[["Id"]]

In [ ]:
tdf.insert(1, "SalePrice", f_predictions)

In [ ]:
tdf.to_csv("/content/datasets/MyDrive/datasets/house-prices-advanced-regression-techniques/predictions.csv", index=False)